<a href="https://colab.research.google.com/github/TiffanyN22/BMES-EEG-Project/blob/main/Copy_of_EEGWaves_TensorFlow_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne

In [ ]:
#!unzip /content/drive/MyDrive/EEGWaves/sleep-cassette.zip -d /content/drive/MyDrive/EEGWaves/sleep-cassette

In [ ]:
import mne
import os
import numpy as np

In [ ]:
edf_folder = "/content/drive/MyDrive/EEGWaves/sleep-cassette"
edf_files = [os.path.join(edf_folder, f) for f in os.listdir(edf_folder) if f.endswith(".edf")]

In [ ]:
# Create two dictionaries, one for PSG files and one for Hypnogram files
psg_files = {f[:7]: f for f in edf_files if '-PSG.edf' in f}
hyp_files = {f[:7]: f for f in edf_files if '-Hypnogram.edf' in f}

# Now you can get the PSG file and its corresponding Hypnogram file by the key
for key in psg_files.keys():
    print(psg_files[key], hyp_files[key])

/content/drive/MyDrive/EEGWaves/sleep-cassette/SC4731E0-PSG.edf /content/drive/MyDrive/EEGWaves/sleep-cassette/SC4122EV-Hypnogram.edf


In [ ]:
# Calculate the start and stop times of each epoch
epoch_times = np.arange(raw.times[0], raw.times[-1], 30)

# Create a list to hold our labels
labels = []

# Also create a list to hold our epochs
epochs_data = []

In [ ]:
# Iterate over the start times
for start in epoch_times:
    # Find the annotation that overlaps with this time
    for ann in raw.annotations:
        if ann['onset'] <= start < ann['onset'] + ann['duration']:
            # If we found an overlapping annotation, save the label and the data
            labels.append(ann['description'])
            epoch = raw.copy().crop(tmin=start, tmax=start+30).get_data()
            epochs_data.append(epoch)
            break


In [ ]:
# Convert labels and epochs_data to numpy arrays
labels = np.array(labels)
epochs_data = np.array(epochs_data)

# Check the shapes of our data
print(epochs_data.shape)
print(labels.shape)

(2880, 7, 3001)
(2880,)


In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Reshape epochs_data to 2D so it can be processed by StandardScaler
n_epochs, n_channels, n_times = epochs_data.shape
epochs_data = epochs_data.reshape(n_epochs, n_channels * n_times)

# Standardize features
scaler = StandardScaler()
epochs_data = scaler.fit_transform(epochs_data)

# Encode labels
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)

# Print shapes of data and labels to make sure everything is okay
print(epochs_data.shape)
print(labels.shape)


(2880, 21007)
(2880,)


In [ ]:
from sklearn.model_selection import train_test_split

# First, split the data into a training+validation set and a test set
X_temp, X_test, y_temp, y_test = train_test_split(
    epochs_data, labels, test_size=0.2, random_state=42
)

# Then, split the training+validation set into separate training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42  # 0.25 x 0.8 = 0.2
)

# Check the shapes to make sure everything is OK
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)


(1728, 21007)
(1728,)
(576, 21007)
(576,)
(576, 21007)
(576,)


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Determine the number of features
n_features = X_train.shape[1]

# Build the model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(n_features,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(np.unique(y_train)), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, 
          validation_data=(X_val, y_val), 
          epochs=10, 
          batch_size=32)


Epoch 1/10
54/54 [==============================] - 2s 10ms/step - loss: 1.3996 - accuracy: 0.6944 - val_loss: 1.4631 - val_accuracy: 0.7222
Epoch 2/10
54/54 [==============================] - 0s 5ms/step - loss: 1.1254 - accuracy: 0.7118 - val_loss: 1.0817 - val_accuracy: 0.7326
Epoch 3/10
54/54 [==============================] - 0s 5ms/step - loss: 1.0312 - accuracy: 0.7135 - val_loss: 1.0455 - val_accuracy: 0.7326
Epoch 4/10
54/54 [==============================] - 0s 5ms/step - loss: 0.9920 - accuracy: 0.7147 - val_loss: 1.0379 - val_accuracy: 0.7326
Epoch 5/10
54/54 [==============================] - 0s 5ms/step - loss: 0.9722 - accuracy: 0.7153 - val_loss: 1.0692 - val_accuracy: 0.7326
Epoch 6/10
54/54 [==============================] - 0s 5ms/step - loss: 0.9455 - accuracy: 0.7159 - val_loss: 1.0590 - val_accuracy: 0.7292
Epoch 7/10
54/54 [==============================] - 0s 5ms/step - loss: 0.9234 - accuracy: 0.7176 - val_loss: 1.0480 - val_accuracy: 0.7326
Epoch 8/10
54/54 [=

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_accuracy}")

18/18 [==============================] - 0s 3ms/step - loss: 0.9840 - accuracy: 0.7326
Test loss: 0.9840005040168762
Test accuracy: 0.7326388955116272
